Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [159]:
from random import choices, choice, random, randint, sample
from copy import copy

import lab9_lib

In [160]:
GENOME_LENGTH = 50

fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=GENOME_LENGTH)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10101011100001000111011010111110110111101101001001: 23.33%
11110110111101100011000001010011001110011011100000: 7.33%
10110101011100001101110001010101110000010010111011: 9.33%
00000011111011010010111011001110100010110111001100: 15.33%
11101111001111110111111001011011100100000010010110: 19.11%
01110100100100001111101001110010011001111010100000: 23.56%
00000000101000110101101001101101000010001100100011: 17.56%
10110100101011010011100000101111100011000110110111: 23.34%
11000110110111001101110010001001010000100110000000: 9.36%
11101011011000101111000110010101011110001000000001: 15.33%
10


In [161]:
OFFSPRING_SIZE = 30
MUTATION_PROBABILITY = .15
POPULATION_SIZE = 50
TOURNAMENT_SIZE = 4

In [162]:
def tournament_selection(population):
    return max(
        [choice(population) for _ in range(TOURNAMENT_SIZE)], key=lambda i: fitness(i)
    )


def one_cut_xover(ind1, ind2):
    cut_point = randint(0, GENOME_LENGTH - 1)
    offspring = ind1[:cut_point] + ind2[cut_point:]
    return offspring


def flip_random_bit(ind):
    offspring = copy(ind)
    pos = randint(0, GENOME_LENGTH - 1)
    offspring[pos] = int(not offspring[pos])
    return offspring


def swap_two_bits(ind):
    offspring = copy(ind)
    pos1, pos2 = tuple(choices(range(0, GENOME_LENGTH), k=2))
    offspring[pos1] = ind[pos2]
    offspring[pos2] = ind[pos1]
    return offspring


def scramble(ind):
    offspring = copy(ind)
    pos1 = randint(0, GENOME_LENGTH - 1)
    pos2 = randint(pos1, GENOME_LENGTH - 1)
    to_scramble = offspring[pos1:pos2]
    permutation = sample(to_scramble, len(to_scramble))
    offspring[pos1:pos2] = permutation
    return offspring


def inversion(ind):
    offspring = copy(ind)
    pos1 = randint(0, GENOME_LENGTH - 1)
    pos2 = randint(pos1, GENOME_LENGTH - 1)
    offspring[pos1:pos2] = list(reversed(ind[pos1:pos2]))
    return offspring

In [163]:
population = [choices([0, 1], k=GENOME_LENGTH)] * POPULATION_SIZE


def evolution(population, parent_selection, crossover, mutation, generations):
    best_fitness = float("-inf")
    consecutive_no_improvement = 0
    max_consecutive_no_improvement = generations / 10

    for generation in range(generations):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = parent_selection(population)
                o = mutation(p)
            else:
                # xover # add more xovers
                p1 = parent_selection(population)
                p2 = parent_selection(population)
                o = crossover(p1, p2)
            offspring.append(o)

        population.extend(offspring)
        population.sort(key=lambda i: fitness(i), reverse=True)
        population = population[:POPULATION_SIZE]
        print(fitness(population[0]), population[0])

        if fitness(population[0]) > best_fitness:
            best_fitness = fitness(population[0])
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement >= max_consecutive_no_improvement:
            print(f"Converged at generation {generation}")
            return


print(evolution(population, tournament_selection, one_cut_xover, scramble, 1000))

0.39333579999999996 [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
0.39333579999999996 [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
0.39333579999999996 [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
0.473358 [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
0.473358 [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
0.473358 [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
0.47335